In [3]:
import pandas as pd
dates = pd.read_csv('movie_lens/dates.csv')
movies = pd.read_csv('movie_lens/movies.csv')
ratings1 = pd.read_csv('movie_lens/ratings1.csv')
ratings2 = pd.read_csv('movie_lens/ratings2.csv')

dates['date'] = pd.to_datetime(dates['date'])
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
ratings = ratings.drop_duplicates(ignore_index=True)
ratings_dates = pd.concat([ratings, dates], axis=1)


___МЕТОД ОБЪЕДИНЕНИЯ JOIN___  
Для объединения двух таблиц по индексам используется метод DataFrame __join()__. Однако данный метод можно применить и для того, чтобы объединить таблицы по ключевому столбцу (в нашем случае это movieId).  

Основные параметры метода __join()__  
- __other__ — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».  
- __how__ — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right' (right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left'.  
- __on__ — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».  
- __lsuffix и rsuffix__ — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

Если использовать метод __join()__ «в лоб» (без указания ключевого столбца), то объединение произойдёт, как и задумано — по индексам двух таблиц согласно установленному типу объединения.

In [6]:
# Проверим это, объединив таблицы типом left.
# Так как в наших таблицах есть одноимённые столбцы,
# установим один из суффиксов, чтобы избежать ошибки
joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='right'
)
display(joined_false)

,userId,movieId,rating,date,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...
9737,64,3481,4.0,2006-10-22 12:37:45,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,64,3489,3.0,2006-10-22 23:28:09,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,64,3499,4.5,2006-10-22 23:26:41,193585,Flint (2017),Drama
9740,64,3510,3.0,2006-10-22 23:27:26,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


Чтобы совместить таблицы по ключевому столбцу с помощью метода __join()__, необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать с помощью метода __set_index()__. Также необходимо указать название ключа в параметре on.

In [18]:
joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined)

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi


___МЕТОД ОБЪЕДИНЕНИЯ MERGE___  
Аналогично предыдущему, метод __merge()__ предназначен для слияния двух таблиц по ключевым столбцам или по индексам. Однако, в отличие от __join()__, метод __merge()__ предлагает более гибкий способ управления объединением, благодаря чему является более популярным.

Основные параметры метода ___merge()___  
- __right__ — присоединяемая таблица. По умолчанию она является «правой»  
- __how__ — параметр типа объединения. По умолчанию принимает значение 'inner'  
- __on__ — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную  
- __left_on__ — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы  
- __right_on__ — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы  
- __lsuffix и rsuffix__ — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах

In [7]:
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
display(merged.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [8]:
# проверим совпадает ли число строк в merged и ratings_dates
print(merged.shape[0])
print(ratings_dates.shape[0])

100836
100836


In [9]:
merged2 = ratings_dates.merge(
    movies,
    on='movieId',
    how='outer'
)
print(merged2.shape[0])
display(merged2.tail())
# объединили полностью, пропуски заполнены ерундой
# не все фильмы получили оценку

100854


,userId,movieId,rating,date,title,genres
100849,NaN,30892,NaN,NaT,In the Realms of the Unreal (2004),Animation|Documentary
100850,NaN,32160,NaN,NaT,Twentieth Century (1934),Comedy
100851,NaN,32371,NaN,NaT,Call Northside 777 (1948),Crime|Drama|Film-Noir
100852,NaN,34482,NaN,NaT,"Browning Version, The (1951)",Drama
100853,NaN,85565,NaN,NaT,Chalet Girl (2011),Comedy|Romance


In [10]:
merge_ratings = ratings1.merge(ratings2, how='outer')
display(merge_ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


Обратите внимание, что при использовании метода __merge()__ для склейки двух таблиц у нас автоматически пропали дубликаты, которые мы видели при использовании метода concat(). Это особенность метода __merge()__ — _автоматическое удаление дублей_.